### CNN with Image Augmentation (CIFAR10)
images of 10 categories: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. In RGB, 32x32 pixels. The state of the art accuracy is 96.53 percent.


In [13]:
# import tensorflow as tf
import numpy as np                                
# from tensorflow.examples.tutorials.mnist import input_data
import keras 
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import (Dense, Dropout, Activation,
                                     Flatten)
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import load_model
import cv2

In [14]:
# #load data
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# img_rows, img_cols , channels= 32,32,3

In [52]:
b  = x_train[0]

b = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
b = b/127.5 -1
b.shape

(32, 32)

In [54]:
np.array_equal(a[0],b)

True

In [81]:
#load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

def gray_tfnorm(arr):
    arr = np.array([[ i/127.5-1 for i in cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)] for i in arr])
    arr = np.expand_dims(arr, axis=-1)
    return arr

x_train = gray_tfnorm(x_train)
x_test = gray_tfnorm(x_test)

print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 1)
(10000, 32, 32, 1)


In [83]:
img_size, channels= 32,1

In [84]:
# set up image augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(x_train)

In [85]:
# labels
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## 如果include_top = True，shape不能自訂

In [8]:
# model = load_model('./vgg_cifar10.h5', compile = False)
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['acc'])

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False,
                  input_shape=(img_size, img_size, channels))

x = Flatten()(vgg_model.output)
# x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
outputs = Dense(num_class, activation='softmax')(x)


model = Model(inputs=vgg_model.inputs, outputs=outputs)


In [80]:
vgg_model.trainable = True

trainable_layer = 3
for layer in vgg_model.layers[:-trainable_layer]:
    layer.trainable = False

for layer in model.layers:
    pass
#     print(layer, layer.trainable)
    
learning_rate = 1e-4
optimizer = keras.optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

NameError: name 'vgg_model' is not defined

In [10]:
# training without augmentation

model.fit(x_train, y_train, batch_size=128, epochs=3, validation_data=(x_test, y_test))

In [11]:
model.save('vgg_cifar10.h5') 